In [53]:
import csv
import numpy as np
import pandas as pd

import os
import sys
import inspect
import time

### STEP 1: GET SCENARIO WITH MOST NUMBER OF FLOODED LOCATION 

In [54]:
file = 'scenario_lookup.tab'

# Small scale
path = '/Users/kyoung/Box Sync/github/run/heuristic_path_zEnumeration0_hnnh0_intensity2_10182020_0_single_1_124_free_z/input/'

# Full scale
# path = '/Users/kyoung/Box Sync/github/run/heuristic_path_zEnumeration1_hnnh1_intensity2_10182020_1_single_1_124_free_z/input/'
df = pd.read_csv(path + file, delimiter='\t')

list_scenarios = list(df['scenario'].unique())
dict_num_flooded = {}
dict_flooded = {}
max_num = 0
for s in list_scenarios:
    df0 = df[df['scenario'] == s]
    dict_flooded[s] = list(df0['sender'])
    if len(df0) > max_num:
        max_num = len(df0)
        max_scenario = s

print(max_scenario, len(list_scenarios))

nw205i2_25-11 25


### STEP 2: FIND STAGING AREA LOCATION FROM THE MAX SCENARIO

In [55]:
# GET MINIMUM SUM OF DISTANCES FROM THE SCENARIO
file = 'input_c_ijv.tab'
df_distance = pd.read_csv(path + file, delimiter='\t')

list_staging = list(df_distance['stagingArea'].unique())
min_distance = 9999999

dict_stagingArea_distance = {}
vehicle = 'v00'
for i in list_staging:
    df0 = df_distance[df_distance['stagingArea'] == i]    
    summation = 0

    for j in dict_flooded[max_scenario]:
        summation = summation + df0[(df0['sender'] == j) & (df0['vehicleType'] == vehicle)]['c_ijv'].values[0]
    
    dict_stagingArea_distance[i] = summation
    
    if summation < min_distance:
        min_distance = summation
        staging_area = i        

In [56]:
print("STAGING AREA WITH MIN. DISTANCE: ", staging_area)

STAGING AREA WITH MIN. DISTANCE:  a001


### STEP 3: GET PARAMETERS 

In [57]:
# DEMANDS
file = 'input_demand_vs.tab'
df_demand = pd.read_csv(path + file, delimiter='\t')
list_scenarios = list(df_demand['scenario'].unique())

# AMBUS CAPACITY
file = 'input_ambulanceCapacity.tab'
df_vehicleCapacity = pd.read_csv(path + file, delimiter='\t')
dict_vehicleCap = dict(zip(df_vehicleCapacity['ambulanceType'], df_vehicleCapacity['capacity']))

# ROUTES(PATHS) WITH STAGING AREA FOUND
file = 'input_c1.tab'
df_routes = pd.read_csv(path + file, delimiter='\t')
df_routes = df_routes[df_routes['stagingArea'] == staging_area]

# RECEIVER CAPACITY
file = 'input_receiverCapacity.tab'
df_capacity = pd.read_csv(path + file, delimiter='\t')

### STEP 4: DEFINE FUNCTIONS FOR ASSIGNING DEMANDS TO VARIABLE X

In [58]:
def choose_routes_new(df_routes, sender):
    df_route_sender = df_routes[(df_routes['sender'] == sender)]
    df_route_sender.reset_index(drop=True)
    return df_route_sender

In [59]:
def divide_ambusCap(dict_vehicleCap, input_demand):
    dict_output = {}
    for key, val in dict_vehicleCap.items():
        if key != 'v00':
            sol = int(input_demand / val)
            remainder = input_demand - (val*sol)
            dict_output[key] = [sol, remainder]
    return dict_output

In [60]:
def get_ambusCap(dict_vehicleCap, dict_solutions, num_ambus, num_demand):
    max_divider = 0
    max_solution = 0

    list_sols = [dict_solutions[key][0] for key in dict_solutions.keys()]
    list_vals = [dict_solutions[key][1] for key in dict_solutions.keys()]
    if max(list_sols) == 0:
        list_solution = ['v00', max(list_vals), num_ambus]
    elif num_ambus == 0:
        list_solution = ['v00', num_demand, num_ambus]
    
    else:
        for key, val in dict_solutions.items():
            if (val[0] != 0) & (dict_vehicleCap[key] >= max_divider):
                max_divider_key = key
                max_divider = dict_vehicleCap[key]
                max_solution = val[0]

        if num_ambus >= max_solution:
            remaining_ambus = num_ambus - max_solution
        else:
            max_solution = num_ambus
            remaining_ambus = num_ambus - max_solution
        
        remainder = num_demand - max_divider * max_solution
        list_solution = [max_divider_key, max_solution, remaining_ambus, remainder]
    return list_solution

In [61]:
def assign_receiver(df_route_by_sender, 
                    patient_type, 
                    num_demand, 
                    input_scenario, 
                    df_capacity, 
                    list_solutions, 
                    dict_vehicleCap,
                    num_ambus):
    
    if patient_type == 'nc':
        
        dict_solutions = divide_ambusCap(dict_vehicleCap, num_demand)
        vehicle_sol = get_ambusCap(dict_vehicleCap, dict_solutions, num_ambus, num_demand)

        input_df_route_sender = df_route_by_sender[df_route_by_sender['vehicleType'] == vehicle_sol[0]]
        input_df_route_sender= input_df_route_sender.sort_values(by='c_v')
        input_df_route_sender.reset_index(drop=True)

        total_ambus_used = vehicle_sol[1]
        demand_remove = dict_vehicleCap[vehicle_sol[0]] * total_ambus_used

    
        for row_num in range(len(input_df_route_sender)):

            this_staging = input_df_route_sender.iloc[row_num]['stagingArea']
            this_vehicle_type = input_df_route_sender.iloc[row_num]['vehicleType']
            this_sender = input_df_route_sender.iloc[row_num]['sender']
            this_receiver = input_df_route_sender.iloc[row_num]['receiver']
            this_capacity = df_capacity.loc[(df_capacity['receiver'] == this_receiver) & (df_capacity['patientType'] == patient_type)]['receiverCapacity'].values[0]
            this_vehicleCap = dict_vehicleCap[this_vehicle_type]

            if vehicle_sol[0] == 'v00':

                if this_capacity >= demand_remove:
                    remaining_capacity = this_capacity - demand_remove
                    sol_string = [this_staging, this_sender, this_receiver, this_staging, this_vehicle_type, patient_type, input_scenario, demand_remove]
                    list_solutions.append(sol_string)
                    df_capacity.loc[(df_capacity['receiver'] == this_receiver) & (df_capacity['patientType'] == patient_type),'receiverCapacity'] = remaining_capacity
                    demand_remove = 0
                    break

                else:
                    sol_string = [this_staging, this_sender, this_receiver, this_staging, this_vehicle_type, patient_type, input_scenario, this_capacity]
                    list_solutions.append(sol_string)
                    demand_remove = demand_remove - this_capacity
                    df_capacity.loc[(df_capacity['receiver'] == this_receiver) & (df_capacity['patientType'] == patient_type),'receiverCapacity'] = 0
            else:        

                if this_capacity >= demand_remove:

                    ambus_used = int(demand_remove / dict_vehicleCap[vehicle_sol[0]])

                    remaining_capacity = this_capacity - demand_remove
                    sol_string = [this_staging, this_sender, this_receiver, this_staging, this_vehicle_type, patient_type, input_scenario, demand_remove]
                    list_solutions.append(sol_string)
                    df_capacity.loc[(df_capacity['receiver'] == this_receiver) & (df_capacity['patientType'] == patient_type),'receiverCapacity'] = remaining_capacity
                    demand_remove = 0
                    num_ambus = num_ambus - ambus_used
                    total_ambus_used = total_ambus_used - ambus_used

                    if total_ambus_used == 0:
                        demand_leftover = vehicle_sol[3]
                        sol_string = [this_staging, this_sender, this_receiver, this_staging, 'v00', patient_type, input_scenario, demand_leftover]
                        list_solutions.append(sol_string)
                        remaining_capacity = this_capacity - demand_leftover
                        df_capacity.loc[(df_capacity['receiver'] == this_receiver) & (df_capacity['patientType'] == patient_type),'receiverCapacity'] = remaining_capacity
                        break

                else:

                    for i in range(vehicle_sol[1]):
                        this_lot = dict_vehicleCap[vehicle_sol[0]] * (i+1)

                        if this_capacity < this_lot:
                            ambus_used = i
                            demand_by_ambus = dict_vehicleCap[vehicle_sol[0]] * i
                            break

                    sol_string = [this_staging, this_sender, this_receiver, this_staging, this_vehicle_type, patient_type, input_scenario, demand_by_ambus]
                    list_solutions.append(sol_string)
                    remaining_capacity = this_capacity - demand_by_ambus
                    df_capacity.loc[(df_capacity['receiver'] == this_receiver) & (df_capacity['patientType'] == patient_type),'receiverCapacity'] = remaining_capacity
                    num_ambus = num_ambus - ambus_used
                    total_ambus_used = total_ambus_used - ambus_used
                    demand_remove = demand_remove - demand_by_ambus

                    if total_ambus_used == 0:
                        demand_leftover = vehicle_sol[3]
                        sol_string = [this_staging, this_sender, this_receiver, this_staging, 'v00', patient_type, input_scenario, demand_leftover]
                        list_solutions.append(sol_string)
                        remaining_capacity = this_capacity - demand_leftover
                        df_capacity.loc[(df_capacity['receiver'] == this_receiver) & (df_capacity['patientType'] == patient_type),'receiverCapacity'] = remaining_capacity
                        break

    elif patient_type == 'c':
        
        input_df_route_sender = df_route_by_sender[df_route_by_sender['vehicleType'] == 'v00']
        input_df_route_sender= input_df_route_sender.sort_values(by='c_v')
        input_df_route_sender.reset_index(drop=True)
        demand_remove = num_demand

        for row_num in range(len(input_df_route_sender)):
        
            this_staging = input_df_route_sender.iloc[row_num]['stagingArea']
            this_vehicle_type = input_df_route_sender.iloc[row_num]['vehicleType']
            this_sender = input_df_route_sender.iloc[row_num]['sender']
            this_receiver = input_df_route_sender.iloc[row_num]['receiver']
            this_capacity = df_capacity.loc[(df_capacity['receiver'] == this_receiver) & (df_capacity['patientType'] == patient_type)]['receiverCapacity'].values[0]
            this_vehicleCap = dict_vehicleCap[this_vehicle_type]
        
            if this_capacity >= demand_remove:
                remaining_capacity = this_capacity - demand_remove
                sol_string = [this_staging, this_sender, this_receiver, this_staging, this_vehicle_type, patient_type, input_scenario, demand_remove]
                list_solutions.append(sol_string)
                df_capacity.loc[(df_capacity['receiver'] == this_receiver) & (df_capacity['patientType'] == patient_type),'receiverCapacity'] = remaining_capacity
                demand_remove = 0
                break

            else:
                sol_string = [this_staging, this_sender, this_receiver, this_staging, this_vehicle_type, patient_type, input_scenario, this_capacity]
                list_solutions.append(sol_string)
                demand_remove = demand_remove - this_capacity
                df_capacity.loc[(df_capacity['receiver'] == this_receiver) & (df_capacity['patientType'] == patient_type),'receiverCapacity'] = 0
        
    return [df_capacity, list_solutions, num_ambus]

### STEP 5: TEST THE FUNCTIONS

In [62]:
## INITIALIZE PARAMETERS

# PARAMETER 1 : RECEIVER CAPACITY
file = 'input_receiverCapacity.tab'
df_capacity = pd.read_csv(path + file, delimiter='\t')
df_capacity.head()

# PARAMETER 2: AMBUS CAPACITY
file = 'input_ambusMax.tab'
with open(path+file, newline = '') as input_file:                                                                                          
    game_reader = csv.reader(input_file, delimiter='\t')
    for line in game_reader:
        num_ambus = int(line[0])

# INPUT PARAMETERS
dict_vehicleCap = {'v00': 1, 'v20': 20}
j = 's318'
df_route_by_sender = choose_routes_new(df_routes, j)
patient_type = 'nc'
num_demand = 4
input_scenario = 'w205i2_25-1'
df_capacity = df_capacity
list_solutions = []
dict_vehicleCap = dict_vehicleCap
num_ambus = 16

In [63]:
output = assign_receiver(df_route_by_sender, 
                         patient_type, 
                         num_demand, 
                         input_scenario, 
                         df_capacity, 
                         list_solutions, 
                         dict_vehicleCap,
                         num_ambus)

In [64]:
output[1], output[2]

([], 16)

# START ENUMERATION 

In [65]:
time_enumeration_start = time.time()
list_patient_types = ['nc', 'c']
list_solutions = []

# START THE ALGORITHM
for s in list_scenarios:
    print('SCENARIO: ', s)
    
    # INITIALIZE PARAMETERS
    # PARAMETER 1 : RECEIVER CAPACITY
    file = 'input_receiverCapacity.tab'
    df_capacity = pd.read_csv(path + file, delimiter='\t')

    # PARAMETER 2: AMBUS CAPACITY
    file = 'input_ambusMax.tab'
    with open(path+file, newline = '') as input_file:                                                                                          
        game_reader = csv.reader(input_file, delimiter='\t')
        for line in game_reader:
            num_ambus = int(line[0])

    
    df1 = df_demand[df_demand['scenario'] == s]
    list_senders = list(df1['sender'].unique())
    print('    NUMBER OF SENDERS: ', len(list_senders))

    for j in list_senders:
        df2 = df1[df1['sender'] == j]
        
        # 5.1: Satisfy demand of non-critical patients first
        for p in list_patient_types:
            num_demand = df2[df2['patientType'] == p]['demand'].values[0]
            
            if num_demand == 0:
                continue

            else:
                df_route_by_sender = choose_routes_new(df_routes, j)
                output = assign_receiver(df_route_by_sender, p, num_demand, s, df_capacity, list_solutions, dict_vehicleCap, num_ambus)
                df_capacity = output[0]
                list_solutions = output[1]
                num_ambus = output[2]

time_enumeration_end = time.time()


SCENARIO:  nw205i2_25-10
    NUMBER OF SENDERS:  27
SCENARIO:  nw205i2_25-11
    NUMBER OF SENDERS:  29
SCENARIO:  nw205i2_25-12
    NUMBER OF SENDERS:  29
SCENARIO:  nw205i2_25-13
    NUMBER OF SENDERS:  29
SCENARIO:  nw205i2_25-14
    NUMBER OF SENDERS:  28
SCENARIO:  nw205i2_25-15
    NUMBER OF SENDERS:  29
SCENARIO:  nw205i2_25-9
    NUMBER OF SENDERS:  27
SCENARIO:  wnw205i2_25-8
    NUMBER OF SENDERS:  27
SCENARIO:  nnw205i2_25-22
    NUMBER OF SENDERS:  14
SCENARIO:  nnw205i2_25-23
    NUMBER OF SENDERS:  14
SCENARIO:  nnw205i2_25-24
    NUMBER OF SENDERS:  15
SCENARIO:  nnw205i2_25-25
    NUMBER OF SENDERS:  14
SCENARIO:  nw205i2_25-16
    NUMBER OF SENDERS:  23
SCENARIO:  nw205i2_25-17
    NUMBER OF SENDERS:  19
SCENARIO:  nw205i2_25-18
    NUMBER OF SENDERS:  19
SCENARIO:  nw205i2_25-19
    NUMBER OF SENDERS:  16
SCENARIO:  nw205i2_25-20
    NUMBER OF SENDERS:  16
SCENARIO:  nw205i2_25-21
    NUMBER OF SENDERS:  15
SCENARIO:  w205i2_25-1
    NUMBER OF SENDERS:  8
SCENARIO:  w

In [66]:
print("ENUMERATION TIME: ", time_enumeration_end - time_enumeration_start)
print(len(list_solutions))

list_solutions_final = []
for sols in list_solutions:
    if sols[7] == 0:
        continue
    else:
        list_solutions_final.append(sols)

print(len(list_solutions_final))

ENUMERATION TIME:  8.145691871643066
1593
1058


In [67]:
cols = ['staging', 'sender', 'receiver', 'staging1', 'vehicleType', 'patientType', 'scenario', 'value']
df_solution = pd.DataFrame(list_solutions_final, columns = cols)

In [68]:
list_scenarios = list(df_solution['scenario'].unique())
list_vehicle_types = list(df_solution['vehicleType'].unique())

dict_vehicle_solution = {}
max_vehicles = {}
for vehicle in list_vehicle_types:

    vehicle_capacity = dict_vehicleCap[vehicle]
    df0 = df_solution[df_solution['vehicleType'] == vehicle]
    list_vehicles_used = []

    for s in list_scenarios:
        df1 = df0[df0['scenario'] == s]
        
        num_v = 0
        for row in range(len(df1)):
            vehicle_used = int(df1.iloc[row]['value'] / vehicle_capacity)
            num_v = num_v + vehicle_used
        
        list_vehicles_used.append(num_v)
        
    dict_vehicle_solution[vehicle] = list_vehicles_used
    max_vehicles[vehicle] = max(list_vehicles_used)

In [69]:
max_vehicles

{'v20': 16, 'v00': 1564}

# CALCULATE OBJECTIVE VALUE 

In [70]:
# Calculate solution

# fixed_cost
file = 'input_openingCost.tab'
df_fixed_cost = pd.read_csv(path + file, delimiter='\t')
fixed_cost = df_fixed_cost[df_fixed_cost['stagingArea'] == staging_area]['openingCost'].values[0]


# resource_cost
file = 'input_c_v.tab'
df_resource_cost = pd.read_csv(path + file, delimiter='\t')
dict_resource_cost = dict(zip(df_resource_cost['vehicleType'], df_resource_cost['c_v']))
resource_cost = 0
for key, value in dict_resource_cost.items():
    temp_resource_cost = value * max_vehicles[key]
    resource_cost = resource_cost + temp_resource_cost


# operation_cost
file = 'input_c1.tab'
df_routes = pd.read_csv(path + file, delimiter='\t')
df_routes = df_routes[df_routes['stagingArea'] == staging_area]

file = 'input_probability.tab'
df_probability = pd.read_csv(path + file, delimiter='\t')
dict_probability = dict(zip(df_probability['scenario'], df_probability['probability']))

list_scenarios = list(df_solution['scenario'].unique())
operating_cost = 0
for s in list_scenarios:
    print(s)
    dff = df_solution[df_solution['scenario'] == s]
    
    scenario_sum = 0
    for row in range(len(dff)):
        i = dff.iloc[row]['staging']
        j = dff.iloc[row]['sender']
        k = dff.iloc[row]['receiver']
        v = dff.iloc[row]['vehicleType']
        p = dff.iloc[row]['patientType']
        x = dff.iloc[row]['value']
        c_v = df_routes[(df_routes['stagingArea'] == i) & (df_routes['sender'] == j) & (df_routes['receiver'] == k) & (df_routes['vehicleType'] == v)]['c_v'].values[0]
        
        row_sum = dict_probability[s] * x * c_v
        scenario_sum = scenario_sum + row_sum
    operating_cost = operating_cost + scenario_sum

# Find total objective value
objective_value = fixed_cost + resource_cost + operating_cost
print(objective_value)

nw205i2_25-10
nw205i2_25-11
nw205i2_25-12
nw205i2_25-13
nw205i2_25-14
nw205i2_25-15
nw205i2_25-9
wnw205i2_25-8
nnw205i2_25-22
nnw205i2_25-23
nnw205i2_25-24
nnw205i2_25-25
nw205i2_25-16
nw205i2_25-17
nw205i2_25-18
nw205i2_25-19
nw205i2_25-20
nw205i2_25-21
w205i2_25-1
wnw205i2_25-2
wnw205i2_25-3
wnw205i2_25-4
wnw205i2_25-5
wnw205i2_25-6
wnw205i2_25-7
684793.44


### STEP 6: SANITY CHECK

In [71]:
# SEE IF THE SUM OF X VARIABLES MATCHES WITH DEMAND PARAMETER
# dataframes to compare: df_solution vs. df_demand

In [72]:
# DEMANDS
file = 'input_demand_vs.tab'
df_demand = pd.read_csv(path + file, delimiter='\t')
list_scenarios = list(df_demand['scenario'].unique())
df_demand.head()

,sender,patientType,scenario,demand
0,s001,c,nw205i2_25-10,55
1,s001,nc,nw205i2_25-10,222
2,s001,c,nw205i2_25-11,55
3,s001,nc,nw205i2_25-11,222
4,s001,c,nw205i2_25-12,55


In [73]:
error = 0
list_errors = []
for s in list(df_solution['scenario'].unique()):
    print(s)
    df0 = df_solution.loc[df_solution['scenario'] == s]
    
    for j in list(df0['sender'].unique()):
        
        df1 = df0.loc[df0['sender'] == j]
        
        for p in list(df1['patientType'].unique()):
            patient_transferred = sum(df1.loc[df1['patientType'] == p]['value'].values)
            patient_demand = sum(df_demand.loc[(df_demand['scenario'] == s) & (df_demand['sender'] == j) & (df_demand['patientType'] == p)]['demand'].values)
            if patient_transferred != patient_demand:
                error = error + 1
                list_errors.append([s, j, p])

print(error)
print(list_errors)

nw205i2_25-10
nw205i2_25-11
nw205i2_25-12
nw205i2_25-13
nw205i2_25-14
nw205i2_25-15
nw205i2_25-9
wnw205i2_25-8
nnw205i2_25-22
nnw205i2_25-23
nnw205i2_25-24
nnw205i2_25-25
nw205i2_25-16
nw205i2_25-17
nw205i2_25-18
nw205i2_25-19
nw205i2_25-20
nw205i2_25-21
w205i2_25-1
wnw205i2_25-2
wnw205i2_25-3
wnw205i2_25-4
wnw205i2_25-5
wnw205i2_25-6
wnw205i2_25-7
0
[]
